In [1]:
%reload_ext autoreload
%autoreload
%matplotlib inline
import os, json,glob,sys,io
from types import SimpleNamespace
from experiment import run_model
from eval import calculate_stats, eval_model
import pickle
from datetime import datetime 
import torch
from torch import nn
from transformers import BertTokenizer
from model.MedClinical import Biobert_fc 
from sklearn.metrics import classification_report, f1_score
from util.tools  import load_config
from util.DataLoader  import *
from pathlib import Path, PureWindowsPath, PurePosixPath
import traceback
from matplotlib import pyplot as plt
import pandas as pd

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = "cpu"

In [3]:
config_folder = "config/"
config = load_config(config_folder)

In [4]:
dataprocessor = MultiClassificationProcessor()
test_dataloader, dev_data_len, dev_num_labels, dev_num_train_optimization_steps, all_dev_label_ids = dataprocessor.get_data_loader(config,source='test')    

INFO:root:***** Running training *****
INFO:root:  Num examples = 50010
INFO:root:  Batch size = 24
INFO:root:  Num steps = 20830


In [5]:
def save_missed_cases_to_file(file_name, preds, label_ids, inputs):
    tokenizer = BertTokenizer.from_pretrained('bert-base-cased', do_lower_case=False)
    missed_cases = []
    for i in range(0,len(label_ids)):
        if label_ids[i] !=  preds[i]:
             missed_cases.append([ file_name, preds[i],  label_ids[i] , " ". join (tokenizer.convert_ids_to_tokens(inputs[i])) ])

    #Save into a file
    missed_cases_file = config.programsettings.REPORTS_DIR +'test_results_' + str(datetime.now()).replace(":", "_").replace(".", "_") + ".pkl"
    with open(missed_cases_file, "wb") as f:
        pickle.dump(missed_cases, f)  
        


In [6]:
models_folder_name = config.programsettings.OUTPUT_DIR
data_folder = Path(models_folder_name)
list_model_files = list(data_folder.glob('**/*.bin'))
num_labels = 9

f1_score_list = []
for model_file in list_model_files:
    try:
        print("\n Model file: ", model_file)
        model = torch.load(model_file)
#         model = nn.DataParallel(model)
        x =model.eval()
        test_inputs, test_preds, test_labels, test_loss = eval_model( config, model, test_dataloader, device, num_labels)  
        print(classification_report(test_labels,test_preds ))
        print("\n \n \n")
        f1_value = f1_score(test_labels, test_preds,average='macro')
        f1_score_list.append([model_file,f1_value ])
        file_name = model_file.__str__().replace(".", "_") 
        save_missed_cases_to_file( file_name, test_preds, test_labels, test_inputs)
        
    except:
#         print("Failed due to cpu & GPU incompatibility")
     # printing stack trace 
        traceback.print_exc()         

print("\n \n \n")





 Model file:  outputs\re\BERT_Sequence2020-11-16 05_43_28_786014.bin



              precision    recall  f1-score   support

           0       0.95      0.97      0.96      3464
           1       0.97      0.97      0.97      3723
           2       0.97      0.98      0.98      4274
           3       0.99      0.98      0.98      4090
           4       0.93      0.95      0.94       420
           5       0.97      0.95      0.96      3024
           6       0.97      0.98      0.97      2802
           7       0.91      0.82      0.86       775
           8       1.00      1.00      1.00     27438

    accuracy                           0.98     50010
   macro avg       0.96      0.96      0.96     50010
weighted avg       0.98      0.98      0.98     50010


 
 


 Model file:  outputs\re\BERT_Sequence2020-11-16 22_21_41_536138.bin



              precision    recall  f1-score   support

           0       0.95      0.97      0.96      3464
           1       0.97      0.98      0.97      3723
           2       0.99      0.97      0.98      4274
           3       0.99      0.98      0.99      4090
           4       0.95      0.95      0.95       420
           5       0.98      0.94      0.96      3024
           6       0.97      0.98      0.98      2802
           7       0.89      0.81      0.85       775
           8       0.99      1.00      1.00     27438

    accuracy                           0.98     50010
   macro avg       0.96      0.95      0.96     50010
weighted avg       0.98      0.98      0.98     50010


 
 


 Model file:  outputs\re\BERT_Sequence2020-11-20 04_03_11_148846.bin



              precision    recall  f1-score   support

           0       0.95      0.97      0.96      3464
           1       0.97      0.97      0.97      3723
           2       0.98      0.98      0.98      4274
           3       0.98      0.98      0.98      4090
           4       0.94      0.96      0.95       420
           5       0.97      0.94      0.96      3024
           6       0.97      0.98      0.97      2802
           7       0.88      0.83      0.85       775
           8       1.00      1.00      1.00     27438

    accuracy                           0.98     50010
   macro avg       0.96      0.96      0.96     50010
weighted avg       0.98      0.98      0.98     50010


 
 


 Model file:  outputs\re\BERT_Sequence2020-11-20 12_33_00_408665.bin



              precision    recall  f1-score   support

           0       0.94      0.97      0.96      3464
           1       0.97      0.98      0.97      3723
           2       0.98      0.98      0.98      4274
           3       0.99      0.98      0.99      4090
           4       0.94      0.95      0.95       420
           5       0.97      0.94      0.96      3024
           6       0.97      0.98      0.97      2802
           7       0.91      0.81      0.86       775
           8       1.00      1.00      1.00     27438

    accuracy                           0.98     50010
   macro avg       0.96      0.95      0.96     50010
weighted avg       0.98      0.98      0.98     50010


 
 


 Model file:  outputs\re\BioBERT_5epochs.bin


...


              precision    recall  f1-score   support

           0       0.96      0.97      0.97      3464
           1       0.98      0.98      0.98      3723
           2       0.99      0.98      0.98      4274
           3       0.99      0.98      0.99      4090
           4       0.96      0.96      0.96       420
           5       0.98      0.94      0.96      3024
           6       0.97      0.98      0.97      2802
           7       0.91      0.86      0.89       775
           8       0.99      1.00      1.00     27438

    accuracy                           0.99     50010
   macro avg       0.97      0.96      0.97     50010
weighted avg       0.99      0.99      0.99     50010


 
 


 Model file:  outputs\re\BioBERT_fc2020-11-22 08_15_02_596134.bin



              precision    recall  f1-score   support

           0       0.95      0.98      0.97      3464
           1       0.98      0.98      0.98      3723
           2       0.98      0.98      0.98      4274
           3       0.99      0.98      0.99      4090
           4       0.94      0.96      0.95       420
           5       0.98      0.95      0.96      3024
           6       0.97      0.98      0.98      2802
           7       0.93      0.83      0.87       775
           8       1.00      1.00      1.00     27438

    accuracy                           0.99     50010
   macro avg       0.97      0.96      0.96     50010
weighted avg       0.99      0.99      0.99     50010


 
 


 Model file:  outputs\re\BioBERT_fc2020-11-22 10_36_33_240569.bin



              precision    recall  f1-score   support

           0       0.96      0.97      0.97      3464
           1       0.97      0.98      0.98      3723
           2       0.98      0.98      0.98      4274
           3       0.99      0.98      0.99      4090
           4       0.95      0.96      0.96       420
           5       0.98      0.94      0.96      3024
           6       0.97      0.98      0.97      2802
           7       0.92      0.84      0.88       775
           8       1.00      1.00      1.00     27438

    accuracy                           0.99     50010
   macro avg       0.97      0.96      0.96     50010
weighted avg       0.99      0.99      0.99     50010


 
 


 Model file:  outputs\re\BioBERT_fc2020-11-22 23_49_51_449255.bin



              precision    recall  f1-score   support

           0       0.96      0.97      0.96      3464
           1       0.98      0.98      0.98      3723
           2       0.99      0.97      0.98      4274
           3       0.99      0.98      0.99      4090
           4       0.94      0.95      0.94       420
           5       0.97      0.95      0.96      3024
           6       0.97      0.98      0.98      2802
           7       0.91      0.84      0.87       775
           8       0.99      1.00      1.00     27438

    accuracy                           0.99     50010
   macro avg       0.97      0.96      0.96     50010
weighted avg       0.99      0.99      0.99     50010


 
 


 Model file:  outputs\re\BioBERT_fc2020-11-23 02_11_58_926877.bin



              precision    recall  f1-score   support

           0       0.96      0.97      0.96      3464
           1       0.97      0.98      0.98      3723
           2       0.98      0.98      0.98      4274
           3       0.98      0.98      0.98      4090
           4       0.94      0.96      0.95       420
           5       0.97      0.95      0.96      3024
           6       0.97      0.98      0.97      2802
           7       0.89      0.86      0.87       775
           8       1.00      1.00      1.00     27438

    accuracy                           0.98     50010
   macro avg       0.96      0.96      0.96     50010
weighted avg       0.98      0.98      0.98     50010


 
 


 
 



In [7]:
pd.set_option('display.max_colwidth', 120)
columns = ['file_namae', 'f1-score']
test_results= pd.DataFrame(f1_score_list, columns = columns)
test_results

,file_namae,f1-score
0,outputs\re\BERT_Sequence2020-11-16 05_43_28_786014.bin,0.959108
1,outputs\re\BERT_Sequence2020-11-16 22_21_41_536138.bin,0.958500
2,outputs\re\BERT_Sequence2020-11-20 04_03_11_148846.bin,0.958429
3,outputs\re\BERT_Sequence2020-11-20 12_33_00_408665.bin,0.958896
4,outputs\re\BioBERT_5epochs.bin,0.962013
5,outputs\re\BioBERT_5epochs_256seq.bin,0.962747
6,outputs\re\BioBERT_fc2020-11-16 00_40_57_659995.bin,0.947514
7,outputs\re\BioBERT_fc2020-11-16 03_37_07_579463.bin,0.962352
8,outputs\re\BioBERT_fc2020-11-16 16_17_55_396529.bin,0.962979
9,outputs\re\BioBERT_fc2020-11-16 20_17_53_374127.bin,0.961864


In [8]:
# print ("F1 Score summary \n")  
# ax = plt.gca()
# ax.get_xaxis().set_visible(False)
# ax.get_yaxis().set_visible(False)
# plt.box(on=None)
# plt.figure(figsize=(14,6))
# the_table = plt.table(cellText=f1_score_list,
#                       colLabels=['file name', 'F1-score'],
#                       loc='center')
# the_table.auto_set_font_size(False)
# the_table.set_fontsize(10)